### Setup

In [6]:
from os import listdir
import os
from summarizer import Summarizer
import torch
import json 
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config

# variable we need to set to get around 
os.environ['KMP_DUPLICATE_LIB_OK']='True'
model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')
device = torch.device('cpu')
transcript_folder = './cleaned_transcripts/'

In [7]:
files = listdir(transcript_folder)
data_arr = []
for file in files:
    file_path = transcript_folder + file
    with open(file_path, 'r') as file_reader:
        data_arr.append(file_reader.read())

## Extractive Summarization

In [8]:
def get_extractive_summary(cleaned_data, extracted_ratio):
    preprocess_text = cleaned_data.strip().replace("\n","")
    extractive_model = Summarizer()
    return extractive_model(preprocess_text, ratio=extracted_ratio)
    
def get_extract_ratio(target, word_count):
    return target / word_count

In [9]:
extractive_summaries = []
for data in data_arr:
    word_count = len(data.split(" "))
    extractive_ratio = get_extract_ratio(90, word_count)
    extractive_short = get_extractive_summary(data, extractive_ratio)
    extractive_summaries.append(extractive_short)

## Abstractive Summarization

In [10]:
def truncate_data(data, target_word_count):
    data_arr = data.split(" ")
    truncated_data = ""
    for i in range(0, target_word_count):
        if(i < len(data_arr)):
            word = data_arr[i]
            truncated_data += word
            truncated_data += " "
        else:
            break
    return truncated_data

In [11]:
def get_abstractive_summary(input_text):
    t5_prepared_Text = "summarize: "+input_text
    tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt").to(device)
    # summmarize 
    summary_ids = model.generate(tokenized_text,
                                        num_beams=6,
                                        no_repeat_ngram_size=2,
                                        min_length=90,
                                        max_length=300,
                                        early_stopping=True)

    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [12]:
abstractive_summaries = []
for data in data_arr:
    truncated_data = truncate_data(data, 350)
    abstractive_summaries.append(get_abstractive_summary(truncated_data))

## Combined Approach

In [13]:
combined_summaries = []
for data in data_arr:
    # Use the word count to calculate the ratio of the original document in the extractive summary
    word_count = len(data.split(" "))
    ratio = get_extract_ratio(450.0, word_count)
    extractive_summary = get_extractive_summary(data, ratio)
    combined_summaries.append(get_abstractive_summary(extractive_summary))

## Cleaning the data to improve results

In looking through the data, we noticed that the first paragraph of each transcript usually only contained "filler" content. We explored removing the first paragraph of each transcript to attempt to reduce the noise going into each model.

In [14]:
cleaned_data_arr = []
for data in data_arr:
    paragraphs = data.split("\n")
    cleaned_data = ""
    # Skipping the first paragraph to remove filler
    for i in range(1, len(paragraphs)):
        cleaned_data += paragraphs[i]
    cleaned_data_arr.append(cleaned_data)

In [15]:
cleaned_extractive_summaries = []
cleaned_abstractive_summaries = []
cleaned_combined_summaries = []

In [16]:
# Cleaned extractive summaries
for data in cleaned_data_arr:
    word_count = len(data.split(" "))
    extractive_ratio = get_extract_ratio(90, word_count)
    extractive_short = get_extractive_summary(data, extractive_ratio)
    cleaned_extractive_summaries.append(extractive_short)
    
# Cleaned Abstractive summaries
for data in cleaned_data_arr:
    truncated_data = truncate_data(data, 350)
    cleaned_abstractive_summaries.append(get_abstractive_summary(truncated_data))
    
# Cleaned combined approach
for data in cleaned_data_arr:
    # Use the word count to calculate the ratio of the original document in the extractive summary
    word_count = len(data.split(" "))
    ratio = get_extract_ratio(410.0, word_count)
    extractive_summary = get_extractive_summary(data, ratio)
    cleaned_combined_summaries.append(get_abstractive_summary(extractive_summary))

# Comparisons

## Sample 1

### Extractive Summarization Sample 1

In [17]:
print("Extractive Summary: ")
print(extractive_summaries[0])
print("\n\n")
print("Trimmed Extractive Summary: ")
print(cleaned_extractive_summaries[0])

Extractive Summary: 
Credera Family Family Happy Friday to you. When we had to make the adjustment to the internship program back in the spring, Ann, Meredith, and Evan in particular did an incredible job of organizing that and the parts that I participated in and they just set a whole agenda, it was just thoughtful an I'm sure it was tremendously valuable and impactful to the to those young people and so well done. But we don't rest on just being like good or wherever at that moment.



Trimmed Extractive Summary: 
A quick video this week just to give you an update on what's going on. When we had to make the adjustment to the internship program back in the spring, Ann, Meredith, and Evan in particular did an incredible job of organizing that and the parts that I participated in and they just set a whole agenda, it was just thoughtful an I'm sure it was tremendously valuable and impactful to the to those young people and so well done. But we don't rest on just being like good or wherev

### Abstractive Summarization Sample 1

In [18]:
print("Abstractive Summary: ")
print(abstractive_summaries[0])
print("\n\n")
print("Trimmed Abstractive Summary: ")
print(cleaned_abstractive_summaries[0])

Abstractive Summary: 
a quick video this week just to give you an update on what's going on. the business is really healthy and so I'm not even going to talk about that today and will go back to normal updates next week. I just want to focus on some of these other things that are extraordinary an right off the top. we hosted our intern Leadership Development Conference, which was part of what we were able to offer interns that did the internship with us.



Trimmed Abstractive Summary: 
the business is really healthy and so I'm not even going to talk about that today and will go back to normal updates next week. we this week hosted our intern Leadership Development Conference, which was part of what we were able to offer the interns that did the internship with us, as well as some of those who chose not to. it was thoughtful and impactful to the to those young people. so well done. Shout out the rebrand that we've all been apart of these week was amazing.


### Combined Approach Summarization Sample 1

In [19]:
print("Combined Approach summary: ")
print(combined_summaries[0])
print("\n\n")
print("Trimmed Combined Approach Summary: ")
print(cleaned_combined_summaries[0])

Combined Approach summary: 
Credera Family Family Happy Friday to you. a quick video this week just to give you an update on what's going on - and so I'm not even going to talk about that today and will go back to normal updates next week. we'll be rolling out new CPM tool in September, which should make the whole process your monthly check-ins with your career coach. no changes to the actual expectations, framework or the expectations this year.



Trimmed Combined Approach Summary: 
we'll be rolling out a new CPM tool in September, which should make the whole process your monthly check-ins with your career coach. no changes to the actual expectations, framework or the expectations this year, or any changes in the sort of promotion process or compensation process. the new tool will be rolled out in september and is expected to be released in january. if you're looking for something new, you can check it out.


## Sample 2

### Extractive Summarization Sample 2

In [20]:
print("Extractive Summary: ")
print(extractive_summaries[1])
print("\n\n")
print("Trimmed Extractive Summary: ")
print(cleaned_extractive_summaries[1])

Extractive Summary: 
Happy Friday to you this time this video is coming from what we call the TV room and I bet that some of you smart people who win Bailey's name Credera contest can guess why we call it the TV room, but we'll find out. Just know that we're very much aware that each of us is going to be on a pretty broad spectrum in terms of our individual comfort with returning to the office, returning to some of the norms we had before, and so all of our Credera reentry plan on that will be very very flexible. We have verbal on a couple of projects at Amgen. Great new win at National Instruments and so I just really want to encourage everyone you know our hard work and tenacity and grit on that is really paying off.



Trimmed Extractive Summary: 
First, I wanted to let you know that we are working on the Credera version of what reopening may look like. Just know that we're very much aware that each of us is going to be on a pretty broad spectrum in terms of our individual comfort w

### Abstractive Summarization Sample 2

In [21]:
print("Abstractive Summary: ")
print(abstractive_summaries[1])
print("\n\n")
print("Trimmed Abstractive Summary: ")
print(cleaned_abstractive_summaries[1])

Abstractive Summary: 
this video is coming from what we call the TV room. it's been a pretty challenging environment for us for the last six weeks or so, but we've been working on the Credera version of what reopening may look like. first, I wanted to let you know that we are following very closely all the different national state local as well as Omnicom oriented sort of guidelines and policies around that. and I'm hoping this will be an encouraging note for you as I give you some updates here so.



Trimmed Abstractive Summary: 
we're working on the Credera version of what reopening may look like. we are following very closely all the different national state local as well as Omnicom oriented sort of guidelines and policies around that, but are coming up with one that's specific to credera. first and foremost health and safety of our team is what we care the most about. i'll be sharing more on that next week or the week after.


### Combined Approach Summarization Sample 2

In [22]:
print("Combined Approach summary: ")
print(combined_summaries[1])
print("\n\n")
print("Trimmed Combined Approach Summary: ")
print(cleaned_combined_summaries[1])

Combined Approach summary: 
we're working on the Credera version of what reopening may look like. the first three or four weeks of this season have stabilized a bit in the last couple of weeks, so congratulations to Kevin McDonald and team past client from Academy there that we’re doing an digital transformation for Cavenders. this week we wrap up so it's awesome when we get to help our clients when it gets celebrated. so thanks to that team for this and enjoy.



Trimmed Combined Approach Summary: 
we're working on the Credera version of what reopening may look like. I want to let you know that it's under underway and specifically the guiding principles that we are working from there is that first and foremost health and safety of our team is what we care the most about. the students received difficult news with a lot of really appreciation and thankfulness Ann showed the tremendous amount of their character and high quality by the way that they responded to it.


## Sample 3

### Extractive Summarization Sample 3

In [23]:
print("Extractive Summary: ")
print(extractive_summaries[2])
print("\n\n")
print("Trimmed Extractive Summary: ")
print(cleaned_extractive_summaries[2])

Extractive Summary: 
Wanted to start out this week talking a bit about the election that's been going on in the United States. And it's been it's been quite nice because it's been always kind of backed by the understanding that we can be honest with each other. In terms of just normal business updates: Our business continues to be in really good shape and really healthy, so October we didn't quite hit the $10,000,000 target that we mentioned a couple weeks ago and thought we were going to get to.



Trimmed Extractive Summary: 
Wanted to start out this week talking a bit about the election that's been going on in the United States. And it's been it's been quite nice because it's been always kind of backed by the understanding that we can be honest with each other. In terms of just normal business updates: Our business continues to be in really good shape and really healthy, so October we didn't quite hit the $10,000,000 target that we mentioned a couple weeks ago and thought we were go

### Abstractive Summarization Sample 3

In [24]:
print("Abstractive Summary: ")
print(abstractive_summaries[2])
print("\n\n")
print("Trimmed Abstractive Summary: ")
print(cleaned_abstractive_summaries[2])

Abstractive Summary: 
if you voted, we're all feeling a bit of anxiety and stress and frustration and emotion on top of everything else that's already going on in our lives and in the year 2020. I wanted for all of us to practice good inclusive leadership, again, regardless of how you vote or how your beliefs or feelings to be respectful of other people and also just appreciate that there are certainly some of your colleagues who are hurting and so do what you can to help those people.



Trimmed Abstractive Summary: 
if you voted, we're all feeling a bit of anxiety and stress and frustration and emotion on top of everything else that's already going on in our lives and in the year 2020. I wanted for all of us to practice good inclusive leadership, again, regardless of how you vote or how your beliefs or feelings to be respectful of other people and also just appreciate that there are certainly some of your colleagues who are hurting and so do what you can to help those people.


### Combined Approach Summarization Sample 3

In [25]:
print("Combined Approach summary: ")
print(combined_summaries[2])
print("\n\n")
print("Trimmed Combined Approach Summary: ")
print(cleaned_combined_summaries[2])

Combined Approach summary: 
our business continues to be in really good shape and really healthy, so October we didn't quite hit the $10,000,000 target that we mentioned a couple weeks ago and thought we were going to get to. November and December look great and are starting to shape up really nicely heading into January in Q1. get some rest and we will see you next week. if you have any questions about the election that's been going on in the u.s., contact us today.



Trimmed Combined Approach Summary: 
our business continues to be in really good shape and really healthy, so October we didn't quite hit the $10,000,000 target that we mentioned a couple weeks ago and thought we were going to get to. November and December look great and are starting to shape up really nicely heading into January in Q1. if you're looking for business updates, it's time to start out talking about the election in the u.s.


## Sample 4

### Extractive Summarization Sample 4

In [26]:
print("Extractive Summary: ")
print(extractive_summaries[3])
print("\n\n")
print("Trimmed Extractive Summary: ")
print(cleaned_extractive_summaries[3])

Extractive Summary: 
Really glad to be giving you a quick update today. So from a business health standpoint we're in great shape. So right at the end of July, which is super exciting and always good for our business and the culture to get that new energy in the building and new talent in the building, so to speak, and so really encouraged by that.



Trimmed Extractive Summary: 
And continue to see tremendous success in win rate on some pretty interesting and big pieces of business and so extremely well done across the board by the team and our leaders. So right at the end of July, which is super exciting and always good for our business and the culture to get that new energy in the building and new talent in the building, so to speak, and so really encouraged by that. Any of those people would be happy to talk with you and really just listen and try to see how we can help you get you connected to the right resources.


### Abstractive Summarization Sample 4

In [27]:
print("Abstractive Summary: ")
print(abstractive_summaries[3])
print("\n\n")
print("Trimmed Abstractive Summary: ")
print(cleaned_abstractive_summaries[3])

Abstractive Summary: 
in the month of July it's really set to be a record month all the way around. high utilization, great revenue and profitability. and continue to see tremendous success in win rate on some pretty interesting and big pieces of business. this week we got pretty big wins at wild type in NRG, Echelon Fitness, teavana I think there are some others that I haven't included there. but really big week there and really strong sales pipeline.



Trimmed Abstractive Summary: 
this week we got pretty big wins at wild type in NRG, Echelon Fitness, teavana. but really big week there and really strong sales pipeline. Looking forward because that's enabled us to do something that was really fun, which is to actually reach back out to all of our campus hires that are supposed to start this fall and offer them the opportunity if they wanted to begin either one or two months early.


### Combined Approach Summarization Sample 4

In [28]:
print("Combined Approach summary: ")
print(combined_summaries[3])
print("\n\n")
print("Trimmed Combined Approach Summary: ")
print(cleaned_combined_summaries[3])

Combined Approach summary: 
this week we got pretty big wins at wild type in NRG, echelon Fitness, teavana. some new campus hires joining us at the end of this month, and so really encouraged by that. that was a fun thing for us to be able to do. I wanted to go ahead and give you that certainty on what it's going to look like. but it can be outside of that category if you have something else you're passionate about, make your contribution.



Trimmed Combined Approach Summary: 
this week we got pretty big wins at wild type in NRG, echelon Fitness, teavana. but really big week there and really strong sales pipeline. so what that will mean is we will begin to have some. some new campus hires joining us at the end of this month. barring any unforeseen challenges, we should get that closed out a week from Monday. you can certainly talk though to your PD or OMC, or your practice leader, your Geo lead. send us in your receipt and we'll


## Sample 5

### Extractive Summarization Sample 5

In [29]:
print("Extractive Summary: ")
print(extractive_summaries[4])
print("\n\n")
print("Trimmed Extractive Summary: ")
print(cleaned_extractive_summaries[4])

Extractive Summary: 
You can see I'm recording from a different venue today. Very, very positive sign for the business and I'm very thankful for that. We helped with some of their online and digital ordering delivery type capabilities and curbside capabilities and you think about the world environment that we're in that has been their saving grace and really helping them to stay afloat and alive during this time and so much so that on Mother's Day they actually had a completely record breaking day with all their online and digital ordering, you know, even a bit overwhelming for their supply chain, and the restaurants themselves were overwhelmed.



Trimmed Extractive Summary: 
You know, at this point May is looking really good in terms of utilization and and ultimately you know close to our revenue target. Very, very positive sign for the business and I'm very thankful for that. Want to celebrate some of the Impact awards and the Junkyard Dog Award. From a Junkyard Dog award standpoint

### Abstractive Summarization Sample 5

In [30]:
print("Abstractive Summary: ")
print(abstractive_summaries[4])
print("\n\n")
print("Trimmed Abstractive Summary: ")
print(cleaned_abstractive_summaries[4])

Abstractive Summary: 
I think we're going to close that gap over the next few days and weeks. it's a very positive sign for the business and I'm very thankful for that. we had wins at 11 or 12 different clients this week, including Novartis, and unilever. i mean, so thankful. so grateful for it. and thankful, thank you for your support. you know, at this point May is looking really good in terms of utilization and ultimately your revenue target.



Trimmed Abstractive Summary: 
we've sold enough new work and I'll hit some wins in a minute. we had wins at I think 11 or 12 different clients this week, but that's really positive sign for the business. i mean, so thankful for that, and grateful for everyone' s efforts into those pursuits. that would be good new business win in any economic climates and especially good in this current one.Quick just kind of follow up on the campus and internship program so that we continue to receive good feedback from the universities.


### Combined Approach Summarization Sample 5

In [31]:
print("Combined Approach summary: ")
print(combined_summaries[4])
print("\n\n")
print("Trimmed Combined Approach Summary: ")
print(cleaned_combined_summaries[4])

Combined Approach summary: 
it's going to go to Anna Grace Franklin, who on earth of Fix and add Orthofix made herself so invaluable to the client that even though they were making some cutbacks across the board, she jumped in. the team later extended on into the summer, and she'd proactively put together kind of the business case in Esso to get herself extended to a new summer - so tremendous job Anna grace and related at orthofix team in general. if people you want to nominate for that, remember to email impact



Trimmed Combined Approach Summary: 
we've sold enough new work and I'll hit some wins in a minute. this week, we had wins at I think 11 or 12 different clients, and so we are just about one month away from when we expect to close that and bring them into the Credera family as well. and all of my interactions with them continue to be encouraging. so two teams and some people to celebrate on the impact side of things, including Kevin King, Grace Levitt, Jane J Carter, Henry J